In [ ]:
from IPython.display import clear_output
import numpy

clear_output()
print("ja")

In [ ]:
# GENERATING ARRAYS
# BAT_SIZE, WIN, WIN, CHANNELS
arr = np.zeros((2, 4, 4, 3))

ind = 1
W, X, Y, Z = arr.shape
for w in range(W):
    ind=1
    for x in range(X):
        for y in range(Y):
            for z in range(Z):
                arr[w, x, y, z] = ind
                ind += 1

# arr = arr.reshape((4, 4, 3))
print(arr.shape)
arr

In [ ]:

# GENERATING FILTERS
# FILTER_NUM, KERNEL, KERNEL, CHANNELS
filt = np.zeros((2, 2, 2, 2))

ind = 0
W, X, Y, Z = filt.shape
for w in range(W):
    for x in range(X):
        ind = 1
        for y in range(Y):
            ind += 1
            for z in range(Z):
                filt[w, x, y, z] = ind

# filt = filt.reshape((2, 2, 2))
print(filt.shape)
filt

In [ ]:
def transconv_valid(input_arr, filters):
    # Function used to get the previous layer activations derivatives
    bat, _, inp_win, num_filt = input_arr.shape
    num_filt, _, kernel, channels = filters.shape

    out_win = inp_win + kernel - 1
    output = np.zeros((bat, out_win, out_win, channels), dtype = np.float32)

    for data in range(bat):
        for filt in range(num_filt):
            for row in range(inp_win):
                for col in range(inp_win):
                    output[data, row:kernel+row, col:kernel+col, :] += filters[filt, :, :, :]*input_arr[data, row, col, filt]
    return output

transconv_valid(filt, arr)

In [ ]:
bat_size, win_size, _, in_channels = arr.shape
kernel = 2

Wq = np.random.randn(in_channels, kernel)*0.01
Wk = np.random.randn(in_channels, kernel)*0.01
Wv = np.random.randn(in_channels, kernel)*0.01
Wo = np.random.randn(kernel, in_channels)*0.01

X_flat = arr.reshape((bat_size, win_size**2, in_channels))

K = np.matmul(X_flat, Wk)
Q = np.matmul(X_flat, Wq)
V = np.matmul(X_flat, Wv)


Attention = np.matmul(np.matmul(Q, K.transpose((0, 2, 1))), V)          # ``````````````````````````````````````````````````

Attention_flat = np.matmul(Attention, Wo)
Attention_weights = Attention_flat.reshape((bat_size, win_size, win_size, in_channels))
output = X + Attention_weights
output

In [ ]:
def tranpose_conv_valid(input_arr, filters):
    _, inp_win, num_filt = input_arr.shape
    num_filt, _, kernel, channels = filters.shape

    out_win = inp_win + kernel - 1
    output = np.zeros((out_win, out_win, channels), dtype = np.float32)

    for filt in range(num_filt):
        for row in range(inp_win):
            for col in range(inp_win):
                output[row:kernel+row, col:kernel+col, :] += filters[filt, :, :, :]*input_arr[row, col, filt]
    return output

def filter_transconv_valid(input_arr, layer_activations, kernel):
    _, inp_win, num_filt = input_arr.shape
    lay_win, _, channels = layer_activations.shape

    output = np.zeros((num_filt, kernel, kernel, channels), dtype = np.float32)

    for filt in range(num_filt):
        for row in range(inp_win):
            for col in range(inp_win):
                output[filt, :, :, :] += layer_activations[row:kernel+row, col:kernel+col, :]*input_arr[row, col, filt]
    return output


# Broadcasting  need reworking
# tran_broad = np.frompyfunc(tranpose_conv_valid, 2, 1)
# tran = np.vectorize(tranpose_conv_valid)
# tran(arr, filt)